In [ ]:
import torch
import numpy as np
import matplotlib.pylab as plt

# 线性模型

最简单最基本的模型，$\hat{y}=x*\omega+b $

定义损失函数为：$loss=\sum(\hat{y}-y)^2=\sum(x*\omega-y)^2$

对不同的$\omega$进行遍历，算出损失，找到最小的$\omega$

In [ ]:
import numpy as np
import matplotlib.pylab as plt

x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

def forward(x):
    return x * w
    
def loss(x, y):
    return (forward(x) - y) ** 2

w_list = []
mse_list = []
for w in np.arange(0.0, 4.0, 0.1):
    # print('w=', w)
    l_sum = 0.0
    for x_val, y_val in zip(x_data, y_data):
        y_pred_val = forward(x_val)
        loss_val = loss(x_val, y_val)
        l_sum += loss_val
        # print('\t', x_val, y_val, y_pred_val, loss_val)
    # print('MSE=', l_sum / len(x_data))
    w_list.append(w)
    mse_list.append(l_sum / 3)
    
plt.plot(w_list, mse_list)
plt.ylabel('loss')
plt.xlabel('w')
plt.show()

# 梯度下降算法

<img src = './Image/Pytorch深度学习实践/梯度下降.png' width = 400 align = right>
对于函数而言，梯度有$\nabla f = \frac{\partial f}{\partial x}$

对于当前的$\omega$，如果梯度大于0，说明误差在增大，否则误差在减小，故下一个取得的$\omega$为：

$\omega = \omega - \alpha * \frac{\partial f}{\partial x}$，其中$\alpha$参数称作学习率(学习率不能太大，否则会直接越过最低点)

梯度下降实际上是一种贪心算法，它得到的是局部区域的一个最优结果，如果损失函数不是一个“凸函数”，可能得不到最好的结果

现实中，太多的局部最优解其实很少见，更常见的是“鞍点”——梯度向量为0的点，在二维平面里，会出现无法继续迭代的现象，在三维即更高的空间里，可能出现从一个面看是最高点，而另一个面是最低点

<img src = './Image/Pytorch深度学习实践/鞍点.jpg' width = 400>

在cost函数中：
$$\frac{\partial cost(\omega)}{\partial \omega} = \frac{\partial}{\partial \omega} \frac{1}{N}\sum_{n=1}^N(x_n*\omega-y_n)^2 = \frac{1}{N}\sum_{n=1}^N2x_n(x_n*w-y_n)
$$
则
$$\omega = \omega - \alpha * \frac{\partial f}{\partial x} = \omega - \alpha * \frac{1}{N}\sum_{n=1}^N2x_n(x_n*w-y_n)$$

In [ ]:
# 梯度下降算法
import matplotlib.pylab as plt

# prepare the training set
x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

# initial guess of weight
w = 1.0


# define the model linear model y = w*x
def forward(x):
    return x * w


#define the cost function MSE
def cost(xs, ys):
    cost = 0
    for x, y in zip(xs, ys):
        y_pred = forward(x)
        cost += (y_pred - y)**2
    return cost / len(xs)


# define the gradient function  gd
def gradient(xs, ys):
    grad = 0
    for x, y in zip(xs, ys):
        grad += 2 * x * (x * w - y)
    return grad / len(xs)


epoch_list = []
cost_list = []
# print('predict (before training)', 4, forward(4))
for epoch in range(100):
    cost_val = cost(x_data, y_data)
    grad_val = gradient(x_data, y_data)
    w -= 0.01 * grad_val  # 0.01 learning rate
    # print('epoch:', epoch, 'w=', w, 'loss=', cost_val)
    epoch_list.append(epoch)
    cost_list.append(cost_val)

# print('predict (after training)', 4, forward(4))
plt.plot(epoch_list, cost_list)
plt.ylabel('cost')
plt.xlabel('epoch')
plt.grid()
plt.show()

## 随机梯度下降算法
不采取整体的cost的梯度，而采取随机一个样本的loss的梯度，避免整体的梯度为0带来的鞍点导致无法更新$\omega$
+ 优点：避免了鞍点，性能好
+ 缺点：整体梯度下降可以采用并行计算最后汇总，而随机算法因为每次都会迭代更新$\omega$，如果要达到同样的训练此数，只能用循环 时间慢

解决方法：
+ 将原数据(batch)分组(mini_batch)，对每组用整体计算cost，然后随机挑选一组的cost

In [ ]:
# 随机梯度下降算法
import matplotlib.pylab as plt

# prepare the training set
x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

# initial guess of weight
w = 1.0


# define the model linear model y = w*x
def forward(x):
    return x * w


#define the loss function
def loss(x, y):
    y_pred = forward(x)
    return (y_pred - y)**2


# define the gradient function  gd
def gradient(x, y):
    return 2 * x * (x * w - y)


epoch_list = []
loss_list = []
# print('predict (before training)', 4, forward(4))
for epoch in range(100):
    for x, y in zip(x_data, y_data):
        l = loss(x, y)
        grad_val = gradient(x, y)
        w -= 0.01 * grad_val  # 0.01 learning rate
        # print('epoch:', epoch, 'w=', w, 'loss=', cost_val)

    epoch_list.append(epoch)
    loss_list.append(l)

# print('predict (after training)', 4, forward(4))
plt.plot(epoch_list, loss_list)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid()
plt.show()

# 反向传播与pytorch初步实现

反向传播的一个简要解析：[梯度反向传播简解](https://zhuanlan.zhihu.com/p/40378224)
<img src = './Image/Pytorch深度学习实践/反向传播.png' width = 300>
结合上图，我们的预测值$y$是一个关于输入的函数：$\hat{y}=f_\omega(\pmb{x})$，梯度下降算法通过对$loss = (\hat{y}-y)^2$求梯度，根据梯度修正参数$\omega$的值直到梯度/导数收敛为0，此时我们可以认为找到了最小的$loss$

问题的关键是如何对每个参数都求得对应的梯度，反向传播的思想为先正向算得中间量(forward过程)，然后利用链式法则，反向计算每个参数的梯度(Back Propagation过程)，前面的中间量可以带入到链式过程中简化运算；

Pytorch中以张量$Tensor(data,grad)$为基本类型(grad也为张量类型)，在构造前向图的时候，梯度信息会自动保留，当前向计算完毕，释放最终结果时，沿着计算图在Tensor里的grad属性即会保留在这次运算中的梯度值

还是以$y=\omega x$为例

In [ ]:
import torch
import matplotlib.pylab as plt

x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

# w即为需要球梯度的张量
w = torch.Tensor([1.0])
w.requires_grad = True


def forword(x):
    # w为Tensor,最后的return会自动类型转换为Tensor
    return x * w

# 每次调用loss函数都会构造出一个计算图
def loss(x, y):
    y_pred = forword(x)
    return (y_pred - y) ** 2

loss_list = []
epoch_list = []

for epoch in range(100):
    for x, y in zip(x_data, y_data):
        l = loss(x, y)
        l.backward()
        w.data = w.data - 0.01 * w.grad.data
        w.grad.data.zero_()
    loss_list.append(l.item())
    epoch_list.append(epoch)

# print(loss_list)
plt.plot(epoch_list, loss_list)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid()
plt.show()

课后作业：假设推测的模型为$\hat{y}=\omega_1x^2+\omega_2x+b$，估计参数(三个方程，三个未知数的求解)

In [ ]:
import torch
import matplotlib.pylab as plt

x_data = [1.0, 2.0, 3.0]
y_data = [2.0, 4.0, 6.0]

# w即为需要球梯度的张量
w1 = torch.Tensor([1.0])
w1.requires_grad = True
w2 = torch.Tensor([1.0])
w2.requires_grad = True
b = torch.Tensor([1.0])
b.requires_grad = True


def forword(x):
    # w为Tensor,最后的return会自动类型转换为Tensor
    return w1 * x * x + w2 * x + b


# 每次调用loss函数都会构造出一个计算图
def loss(x, y):
    y_pred = forword(x)
    return (y_pred - y)**2


l_list = []
epoch_list = []

for epoch in range(100):
    for x, y in zip(x_data, y_data):
        l = loss(x, y)
        l.backward()
        w1.data = w1.data - 0.01 * w1.grad.data
        w2.data = w2.data - 0.01 * w2.grad.data
        b.data = b.data - 0.01 * b.grad.data
        w1.grad.data.zero_()
        w2.grad.data.zero_()
        b.grad.data.zero_()
    l_list.append(l.item())
    epoch_list.append(epoch)

# print(l_list)
print(w1.item(),w2.item(),b.item(),forword(4).item())
plt.plot(epoch_list, l_list)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid()
plt.show()

# pytorch运用

Pytorch的过程为：准备数据集->设计模型->构造损失函数(loss)和优化函数(例如梯度优化，0.01的步长)->循环优化

+ 准备数据

In [ ]:
x_data = torch.tensor([[1.0], [2.0], [3.0]])

y_data = torch.tensor([[2.0], [4.0], [6.0]])

+ 构造模型，Pytorch里的模型都是继承于*torch.nn.Module*



In [ ]:
class LinearModel(torch.nn.Module):
    #调用父类方法
    def __init__(self):
        super().__init__()
        #Linear本身也是一个Module的子类。可以实例化对象
        self.linear = torch.nn.Linear(1, 1)

    # 重载父类forward()函数
    def forward(self, x):
        #由子类实例化出的对象可以使用()重载
        y_pred = self.linear(x)
        return y_pred
    
model = LinearModel()

&emsp;*torch.nn.Linear*的两个参数表示输入的维数和输出的维数例如有n个样本，输入$x$为三维($3\times n$)，输出$\hat{y}$为两维($2\times n$)，则参数矩阵$\omega^T$为$\begin{bmatrix}\cdots\end{bmatrix}_{2\times 3},\hat{y}=\omega^Tx$，另外还有默认参数：*bias=True*, 即 ***torch.nn.Linear(in_features, out_features, bias=True)***

&emsp;对于调用："*y_pred = self.linear(x)*" 此处实际上是重载了类的*()* 符（类似C++里的*void operator()(...)* ），在父类里重写了*__call__* 方法，并由子类继承，可以由实例化对象直接调用

+ 构造损失函数和优化器

In [ ]:
# MSEloss也继承于torch.nn.Module,参数reduction指定最后的loss形式,前两个参数为y'和y
criterion = torch.nn.MSELoss(reduction='sum')

# 来自optim模块,SGD是一个优化器类,第一个参数传递需要求梯度的参数,第二个是步长
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

optimizer1 = torch.optim.Rprop(model.parameters(), lr=0.01)

optimizer2 = torch.optim.Adamax(model.parameters(), lr=0.01)

&emsp;model会在父类里继承$parameters$方法，告知SGD需要反向传播求梯度的参数，此处实际上就是$Linear$类构造函数里传入的参数

+ 开始训练

In [ ]:
epoch_list = []
loss_list = []
loss_list1 = []
loss_list2 = []
for epoch in range(1000):
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)
    loss_list.append(loss.item())
    epoch_list.append(epoch)
    # 使用optimizer会收集参数
    optimizer.zero_grad()
    loss.backward()
    # 自动根据梯度更新权重
    optimizer.step()
    
for epoch in range(1000):
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)
    loss_list1.append(loss.item())
    # 使用optimizer会收集参数
    optimizer1.zero_grad()
    loss.backward()
    # 自动根据梯度更新权重
    optimizer1.step()
    
for epoch in range(1000):
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)
    loss_list2.append(loss.item())
    # 使用optimizer会收集参数
    optimizer2.zero_grad()
    loss.backward()
    # 自动根据梯度更新权重
    optimizer2.step()
    
print('w= ', model.linear.weight.item())
print('b= ', model.linear.bias.item())
print("最后的成品图会受到pytorch自动给定的参数的影响")
print("可以看到,按照不同方式优化的结果,收敛速度不同,并且有可能出现过拟合情况/次数多了反而发散")

#设置中文字符
plt.rcParams['font.family'] = 'SimHei'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.unicode_minus']=False

#为子图设置合适的大小
plt.figure(figsize=(6,9))

#为每个模型构造子图
plt.subplot(3,1,1)
plt.plot(epoch_list, loss_list)
plt.title("SGD优化")
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid()

plt.subplot(3,1,2)
plt.plot(epoch_list, loss_list1)
plt.title("Rprop优化")
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid()

plt.subplot(3,1,3)
plt.plot(epoch_list, loss_list2)
plt.title("Adamax优化")
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid()

#优化间距
plt.tight_layout()
plt.show()

最后的w和b是Tensor类型，需要使用*.item()* 打印数值

# 逻辑斯蒂回归

&emsp;考虑分类问题，将一张手写图片映射到$\mathbb{y}=\{0,1,2,3,4,5,6,7,8,9\}$上，如果考虑用回归的方式处理，注意到这时候输入集和输出集上并没有大小关系的序，而只是不同的类；我们关注的不是某个输入的输出值，而是对每个可能取值的概率

&emsp;pytorch提供了一些数据集：MNIST(手写数字)、CIFAR10(图片分类)

&emsp;在之前的问题中，可以以$y$的大小分为{fail;pass}(二分问题)，我们需要将输入映射到$0\rightarrow 1$的概率输出上(Sigmoid函数/饱和函数)，常用的函数为：$\sigma(x)=\frac{1}{1+e^{-x}}$，$\sigma'(x)=\frac{e^x}{(1+e^x)^2}$特点：有正负极限、单调增、饱和(在正负无穷导数为0，在0处导数max)，对于上述线性模型有：

$$\hat{y}=\sigma(\omega x+b)$$

&emsp;此处$\hat y$表示y取1的概率：$\begin{cases}\begin{align}&\hat y&&P(CLASS=1) \\ &1-\hat y&&P(CLASS=0)\end{align}\end{cases}$，不能继续用$loss=(\hat y - y)^2$计算损失，此处使用交叉熵来衡量离散分布的损失：

$$loss=-\frac{1}{N}\sum_{i=1}^{N}p_{M1}(x=i)logp_{M2}(x=i)\quad p_{M1}为样本的实际值—0或1,p_{M2}为模型输出的概率$$

&emsp;loss加负号使得值越小越好，当二分时，$loss=-(ylog\hat{y}+(1-y)log(1-\hat y))=\begin{cases}\begin{align}&-log\hat y,&&when\quad y=1,\quad \hat y \mathop{\longrightarrow}\limits^{getMax} 1 \quad \\ &-log(1-\hat y),&&when\quad y=0,\quad \hat y \mathop{\longrightarrow}\limits^{getMax} 0 \end{align}\end{cases}$——*BCE Loss*
<img src='./Image/Pytorch深度学习实践/BCE损失.png' width = 300>

*(值得说明的是，之前的线性模型计算的loss其实也不是方差，它是每一个估计量相当于实际值的偏差，而方差为实际值之间的平均分布的衡量)*

In [ ]:
import torch
import matplotlib.pylab as plt

#准备数据集
#-------------------------------------------------------------------------------------------
x_data = torch.tensor([[1.0], [2.0], [3.0]])
y_data = torch.tensor([[0.0], [0.0], [1.0]])


#构建模型
#-------------------------------------------------------------------------------------------
class LogisticRegressionModel(torch.nn.Module):
    #调用父类方法
    def __init__(self):
        super().__init__()
        #Linear本身也是一个Module的子类。可以实例化对象
        self.linear = torch.nn.Linear(1, 1)

    # 重载父类forward()函数
    def forward(self, x):
        #由子类实例化出的对象可以使用()重载
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
model = LogisticRegressionModel()


#设置损失函数和优化器
#-------------------------------------------------------------------------------------------
criterion = torch.nn.BCELoss(reduction='sum')
# 来自optim模块,SGD是一个优化器类,第一个参数传递需要求梯度的参数,第二个是步长
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


#循环优化
#-------------------------------------------------------------------------------------------
loss_list = []
epoch_list = []
for epoch in range(10000):
    y_pred = model.forward(x_data)
    loss = criterion(y_pred, y_data)
    loss_list.append(loss.item())
    epoch_list.append(epoch)
    # 使用optimizer会收集参数
    optimizer.zero_grad()
    loss.backward()
    # 自动根据梯度更新权重
    optimizer.step()


#-------------------------------------------------------------------------------------------
plt.rcParams['font.family'] = 'SimHei'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.unicode_minus'] = False

print('w= ', model.linear.weight.item())
print('b= ', model.linear.bias.item())
print("x = 4.0时的预测为: ", model.forward(torch.Tensor([4.0])).item())
plt.subplot(1, 2, 1)
plt.plot(epoch_list, loss_list)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid()
plt.title("损失函数")

x = np.arange(0, 10, 0.05)
# 利用view将矩阵变成200行一列
x_t = torch.Tensor(x).view((200, 1))
y_t = model(x_t)
# view(-1)表示线性化, 即将矩阵变成1行
y = y_t.data.view(-1).numpy()

plt.subplot(1, 2, 2)
plt.plot(x, y)
plt.ylabel(r'$\hat{y}$')
plt.xlabel('x')
plt.grid()
plt.title("模型预测")

plt.tight_layout()
plt.show()

# 多维特征输入

<img src='./Image/Pytorch深度学习实践/多维输入.png' width = 500>

对于上述的分类任务，每一行称为一个**sample**/样本，每一列称为一个**feature**/特征，在该表中，一个样本有八个特征，对于上述的模型有：

$$
\hat{y}^{(i)}=\sigma(\sum_{n=1}^8\omega_nx_n^{(i)}+b)=\sigma(\pmb w^T\pmb x^{(i)}+b)=\sigma(z^{(i)}),\omega和x均为列向量,z^{(i)}为第i个样本线性计算出的值 
\\
\begin{bmatrix} \hat y^{(1)} \\ \vdots \\ \hat y^{(N)} \end{bmatrix} = 
\begin{bmatrix} \sigma(z^{(1)}) \\ \vdots \\ \sigma(z^{(N)}) \end{bmatrix} = 
\sigma \left( \begin{matrix} z^{(1)} \\ \vdots \\ z^{(N)} \end{matrix} \right),Tensor和numpy一样提供广播
\\ 
\begin{bmatrix} z^{(1)} \\ \vdots \\ z^{(N)} \end{bmatrix}_{N\times 1} = 
\begin{bmatrix}x_1^{(1)}& \cdots & x_8^{(1)}\\
\vdots & \ddots & \vdots \\
x_N^{(1)}& \cdots & x_N^{(8)} \end{bmatrix}_{N\times 8}
\begin{pmatrix}\omega_1 \\ \vdots \\ \omega_8 \end{pmatrix}_{8\times 1} + 
\begin{pmatrix}b \\ \vdots \\ b \end{pmatrix},8维N个样本到1维N个目标的映射
$$

我们的输入为8维，输出仍然为1维，此时使用的变换矩阵维为$[\cdots]_{8\times 1}$我们可以叠加网络层：$[\cdots]_{8\times 2}\times [\cdots]_{2\times 1}$先将8维映射到2维，再将2维映射到1维，为了避免每一个线性变换叠加后仍为线性，为每一个线性变换添加非线性因子——**激活函数**，使得最后为非线性
<img src='./Image/Pytorch深度学习实践/多维映射.png' width = 500>
<img src='./Image/Pytorch深度学习实践/多维线性模型.png' width = 400>

In [ ]:
import numpy as np
import torch
import matplotlib.pylab as plt 

#使用gpu加速，所有相关的运算都要放在gpu上
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ", device)

#准备数据集
#-------------------------------------------------------------------------------------------
# delimiter说明数据间的分隔符
xy = np.loadtxt("./dataset/pytorch深度学习入门/diabetes.csv.gz", delimiter=',', dtype=np.float32)

#取除了最后一列的数据
x_data = torch.tensor(xy[:,:-1])
# x_data = torch.from_numpy(xy[:,:-1])

#取最后一列的数据, 注意到y_data是个二维的矩阵 
y_data = torch.tensor(xy[:,-1:])

#构建模型
#-------------------------------------------------------------------------------------------
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(8, 6)
        self.linear2 = torch.nn.Linear(6, 4)
        self.linear3 = torch.nn.Linear(4, 2)
        self.linear4 = torch.nn.Linear(2, 1)
        self.Relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        x = self.Relu(self.linear1(x))
        x = self.Relu(self.linear2(x))
        x = self.Relu(self.linear3(x))
        x = self.sigmoid(self.linear4(x))
        return x
    
model = Model()
model.to(device)

#设置损失函数和优化器
#-------------------------------------------------------------------------------------------
criterion = torch.nn.BCELoss(reduction='mean')
# 来自optim模块,SGD是一个优化器类,第一个参数传递需要求梯度的参数,第二个是步长
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


#循环优化
#-------------------------------------------------------------------------------------------
loss_list = []
epoch_list = []
x_data, y_data = x_data.to(device), y_data.to(device)
for epoch in range(100000):
    y_pred = model.forward(x_data)
    loss = criterion(y_pred, y_data)
    loss_list.append(loss.item())
    epoch_list.append(epoch)
    # 使用optimizer会收集参数
    optimizer.zero_grad()
    loss.backward()
    # 自动根据梯度更新权重
    optimizer.step()
    
    # 用acc估计模型的精确度
    if epoch%10000 == 9999:
        # torch.where如果满足取第一个否则取第二个, 此处依据y_pred>=0.5进行二分,如果满足则取1, 否则取0
        y_pred_label = torch.where(y_pred>=0.5,torch.tensor([1.0]).to(device),torch.tensor([0.0]).to(device))
        # 统计两个张量里相等元素的个数, 返回值也是张量
        acc = torch.eq(y_pred_label, y_data).sum().item()/y_data.size(0)
        print("loss = ",loss.item(), "acc = ",acc)
    

#-------------------------------------------------------------------------------------------
layer1_weight = model.linear1.weight.data
layer1_bias = model.linear1.bias.data
print("layer1_weight.shape", layer1_weight.shape, "\nlayer1_weight\n", layer1_weight)
print("layer1_bias.shape", layer1_bias.shape, "\nlayer1_bias", layer1_bias)

plt.rcParams['font.family'] = 'SimHei'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.unicode_minus'] = False

# plt.subplot(1, 2, 1)
plt.plot(epoch_list, loss_list)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid()
plt.title("损失函数")
# plt.xticks(np.arange(0,1000000,40000))
plt.show()

# Mini_Batch处理数据集 

上述的*`y_pred = model.forward(x_data)`* 为梯度下降，利用了全部的数据集进行计算，可能会出现鞍点，性能低；如果使用随机梯度下降用一个样本计算会避免鞍点的问题，模型性能比较好（泛化能力），但无法利用并行性，速度慢

使用mini_batch可以综合上述的问题，再追求较好的模型时不至于太慢的训练速度：

-------
三个要点：
+ Epoch
```python
# Training cycle总循环次数`
for epoch in range(training_epochs):`
    # Loop over all batches在所有的batch上进行循环——随机梯度下降`
    for i in range(total_batch):`
```

&emsp;外层设置轮数，内层对batch进行迭代，所有的样本都进行了一轮前馈传播和反向传播，称为一次**epoch**
+ Batch-size

&emsp;一次进行的前馈和反向所用的样本数量

+ Iteration

&emsp;内层循环进行的次数，——mini_batch的个数

------------------

*DataLoader:batch_size=2, shuffle=True* ：两个样本为一个mini_batch，并且对mini_batch进行***shuffle***打乱顺序，得到的Iterator l能执行索引并且知道长度
<img src='./Image/Pytorch深度学习实践/生成mini_batch.png' width = 600>

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class DiabetesDataset(Dataset):
    def __init__(self, filepath):
        xy = np.loadtxt(filepath, delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]  # shape(多少行，多少列) 得到行数即数据集的大小
        self.x_data = torch.from_numpy(xy[:, :-1]).to(device)
        self.y_data = torch.from_numpy(xy[:, -1:]).to(device)

    #魔法方法, 能使用索引来访问元素
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    #魔法方法, 能获取长度
    def __len__(self):
        return self.len 
    
#准备数据集
#-------------------------------------------------------------------------------------------    
dataset = DiabetesDataset("./dataset/pytorch深度学习入门/diabetes.csv.gz")
train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=0)

list(train_loader)

可以看到，加载出来的train_loader每一项都是一个元组，元组里是一个$32样本\times N维$的张量和一个$32样本\times 1维$的张量，即样本特征向取值的映射，这32个样本则作为一个Mini_Batch做一次内循环供后面进行训练

In [ ]:
import numpy as np
import torch
import matplotlib.pylab as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class DiabetesDataset(Dataset):
    def __init__(self, filepath):
        xy = np.loadtxt(filepath, delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]  # shape(多少行，多少列) 得到行数即数据集的大小
        self.x_data = torch.from_numpy(xy[:, :-1]).to(device)
        self.y_data = torch.from_numpy(xy[:, -1:]).to(device)

    #魔法方法, 能使用索引来访问元素
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    #魔法方法, 能获取长度
    def __len__(self):
        return self.len 
    
#准备数据集
#-------------------------------------------------------------------------------------------    
dataset = DiabetesDataset("./dataset/pytorch深度学习入门/diabetes.csv.gz")
train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=0)

#构建模型
#-------------------------------------------------------------------------------------------
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(8, 6)
        self.linear2 = torch.nn.Linear(6, 4)
        self.linear3 = torch.nn.Linear(4, 2)
        self.linear4 = torch.nn.Linear(2, 1)
        self.Relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.Relu(self.linear1(x))
        x = self.Relu(self.linear2(x))
        x = self.Relu(self.linear3(x))
        x = self.sigmoid(self.linear4(x))
        return x


model = Model()
model.to(device)

#设置损失函数和优化器
#-------------------------------------------------------------------------------------------
criterion = torch.nn.BCELoss(reduction='mean')
# 来自optim模块,SGD是一个优化器类,第一个参数传递需要求梯度的参数,第二个是步长
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

loss_list = []
epoch_list = []
for epoch in range(10000):
    losssum = 0.0
    sum = 0
    # 进行两层循环, 如果内存循环mini_batch大小为1, 实际上就是随机梯度下降 
    for data in train_loader:
        inputs, labels = data
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
        optimizer.zero_grad()
        losssum += loss.item()
        sum += 1
        loss.backward()
        optimizer.step() 
    
    losssum /= sum
    if epoch%1000 == 999:
        print(losssum)
    loss_list.append(losssum)
    epoch_list.append(epoch)

plt.rcParams['font.family'] = 'SimHei'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.unicode_minus'] = False

plt.plot(epoch_list, loss_list)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid()
plt.title("损失函数")
plt.show()

# 多分类问题

对于离散分布，有多个取值，一个样本对每个取值都会输出一定概率$\sum p_i=1(p_i>0)$，从中选取概率最大的为预测值；

即我们通过全连接神经网络，将样本的输入（以手写数字识别为例，28\*28=784个像素），映射到分类数（即0-9十个分类）的10个输出，然后将这10个输出处理成概率分布，即***Softmax function***：
$$P(y=i)=\frac{e^{z_i}}{\sum\nolimits_{j=0}^{K-1}e^{z_j}},i\in\{0,...,K-1\}$$

从中选择概率最大的为预测值，然后通过：$LOSS(Y,\hat Y)=-Ylog\hat Y$给出损失值

In [ ]:
import numpy as np
import show_output
y = np.array([1,0,0])
z = np.array([0.2,0.1,-0.1])
y_pred = np.exp(z)/np.exp(z).sum()
loss = (- y * np.log(y_pred)).sum()

loss
y_pred
-1 * np.log(y_pred[0])

## softmax layer
对于前面的层，用Sigmoid激活函数做处理，而对于最后一层则需要特殊处理，使得最后所有输出的值均为正值，并且得到的概率和为1

<img src='./Image/Pytorch深度学习实践/softmax.png' width = 600>

## CrossEntropyLoss损失

交叉熵损失，神经网络最后一层不做非线性变换，而是直接通过CrossEntropyLoss层，得到最后的概率分布
<img src='./Image/Pytorch深度学习实践/CrossEntropyLoss损失.png' width = 600>

In [ ]:
import torch
y = torch.LongTensor([0])
z = torch.Tensor([[0.2,0.1,-0.1]]) # 1个样本*3维特征
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(z,y)
loss

CrossEntropyLoss损失$<=>$Softmax + Log + NLLLoss

$$NLL(log(softmax(input)),target)=-\sum\nolimits_{i=1}^nOneHot(target)_i\times log(softmax(input)_i)$$

In [2]:
import torch
criterion = torch.nn.CrossEntropyLoss()
Y = torch.LongTensor([2,0,1])
Y_pred1 = torch.Tensor([[0.1,0.2,0.9], # 2
                        [1.1,0.1,0.2], # 0
                        [0.2,2.1,0.1]])# 1

Y_pred2 = torch.Tensor([[0.8,0.2,0.3], # 0
                        [0.2,0.3,0.5], # 2
                        [0.2,0.2,0.5]])# 2

l1 = criterion(Y_pred1, Y)
l2 = criterion(Y_pred2, Y)
print("Batch Loss1 =", l1.data, "\nBatch Loss2 =", l2.data)

Batch Loss1 = tensor(0.4966) 
Batch Loss2 = tensor(1.2389)


由下面的程序可以看出，最后得到的CrossEntropyLoss损失，是先对输出Y_pred求softmax后取log，然后挑出其中和样本取值对应的下标的值，然后取负求和求平均（即label的y值是onehot的，而不是带入其本身类别号计算）

In [6]:
import numpy as np
Y = np.array([2,0,1])
Y_pred = np.array([[0.8,0.2,0.3], # 0
                   [0.2,0.3,0.5], # 2
                   [0.2,0.2,0.5]])# 2
Y_softmax =np.exp(Y_pred) / np.sum(np.exp(Y_pred),axis=1).reshape(-1,1) # axis=0按列求和, axis=1按行求和
Y_log = np.log(Y_softmax)
Y_log, -np.array([Y_log[i][Y[i]] for i in range(0,3)]).sum()/len(Y), (Y_log[0][2] + Y_log[1][0] + Y_log[2][1])/3

(array([[-0.76794955, -1.36794955, -1.26794955],
        [-1.23983106, -1.13983106, -0.93983106],
        [-1.2089182 , -1.2089182 , -0.9089182 ]]),
 1.2388996025682044,
 -1.2388996025682044)

In [ ]:
x = np.array([[4,6,8],[3,6,9]])
y = np.array([[2],[3]])
x/y

## 手写数字识别

对于一张图片，它由（通道数channel，宽w，高h）组成，在数据集中，图片表示为$W\times H\times C$，而在pytorch里，图片表示为$C\times W\times H$，需要做一定处理：对于一张灰度图，它的通道数为1，而对于一张彩色图片，它的通道数为3（RGB），在MNIST手写图片数据集中，每个像素都从$(0,255)$取值；

由于神经网络喜欢处理0-1的数据，故先需要将其进行转换，并且，将其映射到标准正态分布上

In [ ]:
from torchvision import transforms
transform = transforms.Compose([
    transforms.ToTensor(),                   # 将数据映射到0-1上
    transforms.Normalize((0.1307,),(0.3801,))# 使用正态分布做归一化处理
])

对于全连接神经网络，处理的是多维特征而不是矩阵，故需要将$N\times1\times28\times$28的矩阵做扁平化处理，将其变为$N\times784$的矩阵，这样它的每一行就可以对应一个特征取值

In [ ]:
import numpy as np
x = np.array([[[1,2,3],[4,5,6],[7,8,9]],         # 第一个3*3的矩阵
              [[10,11,12],[13,14,15],[16,17,18]] # 第二个3*3的矩阵
             ])
x.reshape(-1,9) # 做扁平化处理, 将每个矩阵拉成9维特征的向量

### 获取数据集

In [ ]:
# 获取MNIST数据集
import torchvision
# download = true, 没有该数据集则从网上下载下来
train_set = torchvision.datasets.MNIST(root='./dataset/pytorch深度学习入门/mnist',
                                       train=True,
                                       download=True)
train_set = torchvision.datasets.MNIST(root='./dataset/pytorch深度学习入门/mnist',
                                       train=False,
                                       download=True)

### 具体代码
不做归一化分布处理——`transforms.Normalize((0.1307,), (0.3081,))`，会使得收敛变慢，但最后结果近似

In [8]:
import numpy as np
import torch
import matplotlib.pylab as plt
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
#准备数据集
#-------------------------------------------------------------------------------------------    
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]) # 归一化,均值和方差

# 上一节我们使用DataSet自行构造数据集, 而此处使用torchvision提供好的数据集
# 该datasets里面init, getitem, len魔法函数已实现。
train_dataset = datasets.MNIST(root='./dataset/pytorch深度学习入门/mnist', train=True, download=True, transform=transform) # 训练集
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

test_dataset = datasets.MNIST(root='./dataset/pytorch深度学习入门/mnist', train=False, download=True, transform=transform) # 测试集
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size) # 测试集没有必要做洗牌操作打乱顺序

def trainTest(model, cycles = 10):
    #设置损失函数和优化器
    #-------------------------------------------------------------------------------------------
    criterion = torch.nn.CrossEntropyLoss()
    # 来自optim模块,SGD是一个优化器类,第一个参数接收当前经过梯度递减后的参数,第二个是步长
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
    
    #循环优化
    #-------------------------------------------------------------------------------------------
    
    #封装一轮训练集
    def train(epoch):
        running_loss = 0.0
        for batch_idx, data in enumerate(train_loader,0):
            inputs, target = data
            inputs, target = inputs.to(device), target.to(device)
            optimizer.zero_grad()

            # forword前馈 + backward反馈 + update更新

            #前馈
            outputs = model(inputs)
            loss = criterion(outputs,target)

            #反馈
            loss.backward() # loss进行backward后，实际上是更新模型里权重的梯度

            #优化
            optimizer.step()

            running_loss += loss.item()
            if batch_idx % 300 == 299:
                print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
                running_loss = 0.0

    def test():
        correct = 0
        total = 0
        # 进行验证, 不需要进行梯度
        with torch.no_grad():
            for data in test_loader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, dim=1) # dim = 1按行取最大值, 返回最大值的值和下标的元组
                total += labels.size(0) # 取0表示个数, 取1表示维数
                correct += (predicted == labels).sum().item() # 比较张量, 统计为真的值, 返回个数的标量
        print('accuracy on test set: %d %% ' % (100*correct/total))        

    
    for epoch in range(cycles):
        train(epoch) # 将model作为参数传入可能会有问题
        test()

In [9]:
#构建模型
#-------------------------------------------------------------------------------------------
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)
        self.Relu = torch.nn.ReLU()
 
    def forward(self, x):
        x = x.view(-1, 784)   # -1做扁平化操作
        x = self.Relu(self.l1(x))
        x = self.Relu(self.l2(x))
        x = self.Relu(self.l3(x))
        x = self.Relu(self.l4(x))
        return self.l5(x)    # 最后一层不做激活, 返回线性变换, 用于后续CrossEntropyLoss损失处理
    
model = Net()
model.to(device)
trainTest(model)

[1,   300] loss: 2.247
[1,   600] loss: 1.061
[1,   900] loss: 0.427
accuracy on test set: 90 % 
[2,   300] loss: 0.310
[2,   600] loss: 0.267
[2,   900] loss: 0.225
accuracy on test set: 94 % 
[3,   300] loss: 0.178
[3,   600] loss: 0.167
[3,   900] loss: 0.157
accuracy on test set: 95 % 
[4,   300] loss: 0.128
[4,   600] loss: 0.121
[4,   900] loss: 0.111
accuracy on test set: 96 % 
[5,   300] loss: 0.096
[5,   600] loss: 0.092
[5,   900] loss: 0.089
accuracy on test set: 96 % 
[6,   300] loss: 0.071
[6,   600] loss: 0.073
[6,   900] loss: 0.074
accuracy on test set: 97 % 
[7,   300] loss: 0.062
[7,   600] loss: 0.056
[7,   900] loss: 0.058
accuracy on test set: 97 % 
[8,   300] loss: 0.048
[8,   600] loss: 0.044
[8,   900] loss: 0.048
accuracy on test set: 97 % 
[9,   300] loss: 0.039
[9,   600] loss: 0.037
[9,   900] loss: 0.038
accuracy on test set: 97 % 
[10,   300] loss: 0.030
[10,   600] loss: 0.028
[10,   900] loss: 0.033
accuracy on test set: 97 % 


python的参数传递，当参数非基本类型时，传递的为一个地址形参，可以对地址指向的内容进行修改，而直接对形参地址赋值是不会修改原有值的

In [1]:
x = 1
y = [1, 2, 3]
z = [1, 2, 3]

def changex(x):
    x = 1
def changey(y):
    y[0] = 5
def changez(z):
    z = [2,3,4]
    
changex(x)
changey(y)
changez(z)

x, y, z

(1, [5, 2, 3], [1, 2, 3])

# 卷积神经网络

Convolutional Netural Network —— CNN

之前的全连接神经网络，对于每两个网络层之间的任意两个点，都有相应的权重，如果一个输入大小为100\*100的图像层，映射到100\*100的隐藏层，那么这两个层之间的$\omega$的个数就达到了$100^4$个，考虑用32精度4字节浮点数表示，有$4*10^8$个字节约为400MB，太过庞大；

并且在全连接神经网络中，对于一张图像矩阵，是直接做扁平化处理拉成一维向量，丢失了很多信息

## 卷积核

我们假设一组图像数据的属性为（B,C,W,H）—（batch,channel,宽,高），那么对于该图像的每一个channel，都应该有一个卷积核，对于原图像的每一块卷积核大小的区域，都按位与卷积核做数乘，然后求和，得到该区域经过卷积后的值

一个5\*5大小的图像，经过3\*3的卷积核，变成3\*3的图像：
<img src='./Image/Pytorch深度学习实践/单通道卷积.png' width = 600>
对于每一个输入通道，我们都应该有一个相应的卷积核，对于由这样一组卷积核进行卷积运算得到的结果（$C\times\overline W\times\overline H$）做加法，得到一个输出通道的结果——一张*'C'* 输入多通道图片经过一组*'C'* 个卷积核运算得到一个输出通道
+ 一组卷积操作：
<img src='./Image/Pytorch深度学习实践/一组卷积操作.png' width = 600>

如果我们想得到M个输出通道，那么我们就需要M组卷积核，这样原来的一组图片：$B\times C_{in}\times W_{in}\times H_{in}$变成$B\times\overline C_{out}\times\overline W_{out}\times\overline H_{out}$，最后的$W/H_{out}$由$W/H_{in}$和卷积核大小共同决定：3\*3的卷积核原大小-2，5\*5的卷积核原大小-4
+ m组卷积操作：
<img src='./Image/Pytorch深度学习实践/m组卷积操作.png' width = 600>

In [ ]:
import torch
in_channels,out_channels= 5,10 # 输入通道数, 输出通道数
width,height = 100, 100 # 宽, 高
kernel_size = 3 # 卷积核大小
batch_size = 1
input = torch.randn(batch_size,
                    in_channels,
                    width,
                    height)

conv_layer = torch.nn.Conv2d(in_channels,
                             out_channels,
                             kernel_size=kernel_size)

output = conv_layer(input)
print(input.shape)
print(output.shape)
print(conv_layer.weight.shape) # 10*5*3*3, 10组卷积核, 每组卷积核有5个卷积核=输入通道数, 3*3的卷积核大小

## padding

将图像外层添加一圈（默认）0，使得图像经过3\*3的卷积核后大小不变

In [ ]:
import torch
in_channels,out_channels= 5,10

input = [3,4,6,5,7,
         2,4,6,8,2,
         1,6,7,8,4,
         9,7,4,6,2,
         3,7,5,4,1]

input = torch.Tensor(input).view(1,1,5,5) # b = 1, c = 1, w = 5, h = 5

conv_layer = torch.nn.Conv2d(1, 1, kernel_size=3, padding=1, bias=False)

kernel = torch.Tensor([1,2,3,4,5,6,7,8,9]).view(1, 1, 3, 3) # m/output_channel = 1, c/input_channel = 1, w = 3, h = 3
conv_layer.weight.data = kernel.data # 设置卷积核的权重

output = conv_layer(input)
print(output, output.size())

## 池化层pooling

以MaxPooling为例，设池化层的大小为2（2\*2）,则对每一个通道按2\*2方阵，默认步长stride=2做处理，选取每一个小方阵里最大的数；通过池化层减少数据的大小

## 通过卷积对图像进行处理

对最后的卷积结果，直接扁平化操作(flatten)，将线性特征做全连接映射到10维的输出，做softmax得到概率取值

In [11]:
#构建模型
#-------------------------------------------------------------------------------------------
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5) # 1个输入通道, 10个输出通道的卷积核
        self.conv2 = torch.nn.Conv2d(10,20,kernel_size=5) # 10个输入通道, 20个输出通道的卷积核
        self.pooling = torch.nn.MaxPool2d(2) # 2*2的方阵, 步长为2做池化
        self.l1 = torch.nn.Linear(320, 64)
        self.l2 = torch.nn.Linear(64, 10)
        self.Relu = torch.nn.ReLU()
 
    def forward(self, x):
        batch_size = x.size(0)
        x = self.pooling(self.Relu(self.conv1(x)))
        x = self.pooling(self.Relu(self.conv2(x)))
        x = x.view(batch_size, -1)
        x = self.Relu(self.l1(x))
        return self.l2(x)    # 最后一层不做激活, 返回线性变换, 用于后续CrossEntropyLoss损失处理
    
model = Net()

trainTest(model)

[1,   300] loss: 0.967
[1,   600] loss: 0.233
[1,   900] loss: 0.156
accuracy on test set: 96 % 
[2,   300] loss: 0.119
[2,   600] loss: 0.100
[2,   900] loss: 0.095
accuracy on test set: 97 % 
[3,   300] loss: 0.081
[3,   600] loss: 0.074
[3,   900] loss: 0.068
accuracy on test set: 98 % 
[4,   300] loss: 0.059
[4,   600] loss: 0.061
[4,   900] loss: 0.062
accuracy on test set: 98 % 
[5,   300] loss: 0.051
[5,   600] loss: 0.050
[5,   900] loss: 0.055
accuracy on test set: 98 % 
[6,   300] loss: 0.044
[6,   600] loss: 0.044
[6,   900] loss: 0.047
accuracy on test set: 98 % 
[7,   300] loss: 0.040
[7,   600] loss: 0.038
[7,   900] loss: 0.040
accuracy on test set: 98 % 
[8,   300] loss: 0.037
[8,   600] loss: 0.039
[8,   900] loss: 0.034
accuracy on test set: 98 % 
[9,   300] loss: 0.032
[9,   600] loss: 0.030
[9,   900] loss: 0.035
accuracy on test set: 98 % 
[10,   300] loss: 0.029
[10,   600] loss: 0.030
[10,   900] loss: 0.030
accuracy on test set: 98 % 


# CNN的改进

减少代码冗余：模型中的重复部分可以抽象出来单独作为一个模块，以便重复利用

## Inception Module
将一些卷积核放在一起组成一个模块，经过训练，某些模块的参数会变大（即模型更倾向于选择该模块），从而减少了人为选择模块带来的误差

### 1$\times$1的卷积核：

将图像的三个通道通过1$\times$1的卷积核处理，可以达到对通道进行混合的效果：①例如对RGB三色通道结果$\sum \omega_i=1$进行处理，可以提高某个通道的权重；（每个通道都需要一个1$\times$1的卷积核$\omega_i$）②或者都乘以$\omega=1$的卷积核，相当于求总值

同时，1$\times$1卷积核也可以用于改变原输入的通道数，为后续的$kernel>1$的卷积运算减少运算量

<img src='./Image/Pytorch深度学习实践/1-1卷积核.png' width = 600>

### Concatenate连接

将不同的张量，沿着某一通道进行拼接，例如$2\times4\times4$和$3\times4\times4$的张量拼接到一起，变成$5\times4\times4$的张量，想要对通过不同的卷积核出来的结果进行拼接，首先就要求输出图像的宽高都是一样的，即允许变换的只是图像的通道数Channel（几组卷积核就几个channel），这就要求对图像做处理是要添加padding，使得图像的大小不发生变换

<img src='./Image/Pytorch深度学习实践/Inception.png' width = 400>

可以看到，示例的Inception网络，最后会由四个不同通道数的块Concatenate拼接而成

拼接的张量，必须除了拼接维度上不同以外，其它维度均相同

In [18]:
x = torch.zeros((2,3,2))
y = torch.zeros((2,5,2))
z = torch.cat((x, y), dim = 1)
x, y, z, z.size()

(tensor([[[0., 0.],
          [0., 0.],
          [0., 0.]],
 
         [[0., 0.],
          [0., 0.],
          [0., 0.]]]),
 tensor([[[0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.]],
 
         [[0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.]]]),
 tensor([[[0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.]],
 
         [[0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.],
          [0., 0.]]]),
 torch.Size([2, 8, 2]))

在Concatenate里，（B,C,W,H）中是沿着C进行拼接的，即dim = 1

### 代码实现 

In [17]:
#构建模型, 构造出一个通用模块InceptionA
#-------------------------------------------------------------------------------------------
class InceptionA(torch.nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        
        self.avgpool_1 = torch.nn.AvgPool2d(kernel_size = 3, stride = 1,padding = 1)
        self.avgpool_2 = torch.nn.Conv2d(in_channels, 24, kernel_size = 1)
        
        self.conv1x1 = torch.nn.Conv2d(in_channels, 16, kernel_size = 1)
        
        self.conv5x5_1 = torch.nn.Conv2d(in_channels, 16, kernel_size = 1)
        self.conv5x5_2 = torch.nn.Conv2d(16, 24, kernel_size = 1)
        
        self.conv3x3_1 = torch.nn.Conv2d(in_channels, 16, kernel_size = 1)
        self.conv3x3_2 = torch.nn.Conv2d(16, 24, kernel_size = 1)
        self.conv3x3_3 = torch.nn.Conv2d(24, 24, kernel_size = 1)
        
    def forward(self, x):
        branch_avg = self.avgpool_1(x)
        branch_avg = self.avgpool_2(branch_avg)
        
        branch_1x1 = self.conv1x1(x)
        
        branch_5x5 = self.conv5x5_1(x)
        branch_5x5 = self.conv5x5_2(branch_5x5)
        
        branch_3x3 = self.conv3x3_1(x)
        branch_3x3 = self.conv3x3_2(branch_3x3)
        branch_3x3 = self.conv3x3_3(branch_3x3)
        
        # 最后输出的channel数量为24 + 16 + 24 + 24 = 88
        return torch.cat((branch_avg, branch_1x1, branch_5x5, branch_3x3), dim = 1)

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5) # 1个输入通道, 10个输出通道的卷积核
        self.conv2 = torch.nn.Conv2d(88,20,kernel_size=5) # 10个输入通道, 20个输出通道的卷积核
        self.pooling = torch.nn.AvgPool2d(kernel_size = 3, padding = 1) # 3 * 3的方阵做池化
        
        self.incep1 = InceptionA(in_channels=10)
        self.incep2 = InceptionA(in_channels=20)
        
        self.l1 = torch.nn.Linear(320, 64)
        self.l2 = torch.nn.Linear(64, 10)
        self.Relu = torch.nn.ReLU()
 
    def forward(self, x):
        batch_size = x.size(0)
        x = self.pooling(self.Relu(self.conv1(x)))
        x = self.pooling(self.Relu(self.conv2(x)))
        x = x.view(batch_size, -1)
        x = self.Relu(self.l1(x))
        return self.l2(x)    # 最后一层不做激活, 返回线性变换, 用于后续CrossEntropyLoss损失处理
    
model = Net()
model.to(device)
trainTest(model)

[1,   300] loss: 2.102
[1,   600] loss: 0.468
[1,   900] loss: 0.303
accuracy on test set: 91 % 
[2,   300] loss: 0.233
[2,   600] loss: 0.207
[2,   900] loss: 0.178
accuracy on test set: 94 % 
[3,   300] loss: 0.160
[3,   600] loss: 0.149
[3,   900] loss: 0.138
accuracy on test set: 96 % 
[4,   300] loss: 0.123
[4,   600] loss: 0.121
[4,   900] loss: 0.124
accuracy on test set: 96 % 
[5,   300] loss: 0.107
[5,   600] loss: 0.105
[5,   900] loss: 0.111
accuracy on test set: 96 % 
[6,   300] loss: 0.096
[6,   600] loss: 0.094
[6,   900] loss: 0.094
accuracy on test set: 97 % 
[7,   300] loss: 0.084
[7,   600] loss: 0.082
[7,   900] loss: 0.082
accuracy on test set: 97 % 
[8,   300] loss: 0.078
[8,   600] loss: 0.077
[8,   900] loss: 0.073
accuracy on test set: 97 % 
[9,   300] loss: 0.065
[9,   600] loss: 0.067
[9,   900] loss: 0.067
accuracy on test set: 98 % 
[10,   300] loss: 0.062
[10,   600] loss: 0.061
[10,   900] loss: 0.056
accuracy on test set: 98 % 


## resnet

Residual Net —— 残差网络

在实际运用中，神经网络的效率有可能并不会随着层数的增加而提高——反而有可能下降。其主要原因是，随着神经网络层数的增多，在靠近输入端的梯度随着连乘，可能会趋近于0，而权重的更新公式为：$\omega = \omega - \alpha*grad$，使得权重不会得到有效的更新，最后出现拟合效果差的局面——靠近输入端的权重几乎不变

<img src='./Image/Pytorch深度学习实践/resnet.png' width = 500>

In [18]:
class ResidualBlock(torch.nn.Module):
    def __init__(self, channels):
        super().__init__()
        # 输入通道和输出通道相同
        self.conv1 = torch.nn.Conv2d(channels, channels, kernel_size=3, padding=1) 
        self.conv2 = torch.nn.Conv2d(channels, channels, kernel_size=3, padding=1) 
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        y = self.relu(self.conv1(x))
        y = self.relu(self.conv2(y))
        return self.relu(y + x)

残差网络可以单独作为一个层穿插在神经网络中

### 代码实现

In [20]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5) # 1个输入通道, 10个输出通道的卷积核
        self.conv2 = torch.nn.Conv2d(10,20,kernel_size=5) # 10个输入通道, 20个输出通道的卷积核
        self.pooling = torch.nn.MaxPool2d(2) # 2 * 2的方阵, 步长为2做池化
        
        self.rblock1 = ResidualBlock(channels=10)
        self.rblock2 = ResidualBlock(channels=20)
        
        self.l1 = torch.nn.Linear(320, 64)
        self.l2 = torch.nn.Linear(64, 10)
        self.Relu = torch.nn.ReLU()
 
    def forward(self, x):
        batch_size = x.size(0)
        x = self.pooling(self.Relu(self.conv1(x)))
        x = self.rblock1(x)
        x = self.pooling(self.Relu(self.conv2(x)))
        x = self.rblock2(x)
        x = x.view(batch_size, -1)
        x = self.Relu(self.l1(x))
        return self.l2(x)    # 最后一层不做激活, 返回线性变换, 用于后续CrossEntropyLoss损失处理
    
model = Net()
model.to(device)
trainTest(model)

[1,   300] loss: 0.858
[1,   600] loss: 0.214
[1,   900] loss: 0.150
accuracy on test set: 96 % 
[2,   300] loss: 0.110
[2,   600] loss: 0.097
[2,   900] loss: 0.087
accuracy on test set: 97 % 
[3,   300] loss: 0.077
[3,   600] loss: 0.068
[3,   900] loss: 0.064
accuracy on test set: 98 % 
[4,   300] loss: 0.056
[4,   600] loss: 0.057
[4,   900] loss: 0.055
accuracy on test set: 98 % 
[5,   300] loss: 0.044
[5,   600] loss: 0.051
[5,   900] loss: 0.048
accuracy on test set: 98 % 
[6,   300] loss: 0.039
[6,   600] loss: 0.040
[6,   900] loss: 0.042
accuracy on test set: 98 % 
[7,   300] loss: 0.035
[7,   600] loss: 0.033
[7,   900] loss: 0.039
accuracy on test set: 98 % 
[8,   300] loss: 0.030
[8,   600] loss: 0.032
[8,   900] loss: 0.033
accuracy on test set: 98 % 
[9,   300] loss: 0.027
[9,   600] loss: 0.031
[9,   900] loss: 0.029
accuracy on test set: 98 % 
[10,   300] loss: 0.029
[10,   600] loss: 0.024
[10,   900] loss: 0.026
accuracy on test set: 98 % 


# 循环神经网络

RNN —— Recurrent Neural Network